# Evaporadores
## Determinación del área de intercambio de calor de un evaporador de efecto doble en paralelo

### Esquema del evaporador de efecto doble:


<img src="./Imagenes/Esquema_Evaporadores_1.jpg" height="600" width="600"/>



In [1]:
%matplotlib inline
import numpy as np
from scipy import optimize
from scipy.integrate import ode
import matplotlib.pyplot as mpl
import matplotlib.patches as patches
import random
import math
import pandas as pd
import IPython.core.display as di
from ipywidgets import interact
import notebook
from IPython.display import clear_output, display, HTML, Image,Math, Latex
from IPython.external import mathjax

Cp_vapor=0.46,
Cp_disolucion=1 #Cp en kcal/kg·ºC



def generador_parametros():  # generación de parámetros del problema de un evaporador de doble efecto

    CaudalAlimento = round(random.uniform(20000., 35000.)) #Caudal másico alimento
    W_SolutoAlimento= round(random.uniform(0.01, 0.05), 3) #Fracción másica soluto en la alimentación
    W_SolutoProducto = round(random.uniform(0.35, 0.55), 3) #CFracción másica soluto en la salida del segundo elemento
    Temperatura_Alimento = round(random.uniform(85., 95.), 1) #Temperatura del alimento
    T_SaturacionVapor= round(random.uniform(110., 125.),1) #Temperatura de saturación del vapor de agua
    T_SaturacionCondensadorFinal = round(random.uniform(45., 75.), 1)  # Temperatura de saturación condensador final
    U1= round(random.uniform(2500, 2900.)) #Coeff. Global Efecto 1
    U2= round(random.uniform(1500., 2200.))  #Coeff. Global Efecto 2
    Teb1= round(random.uniform(2, 10.),1) #Elevación del punto de ebullición en el primer efecto
    Teb2= round(random.uniform(5., 15.), 1)  #Elevación del punto de ebullición en el segundo efecto

    return CaudalAlimento,W_SolutoAlimento,W_SolutoProducto,Temperatura_Alimento,T_SaturacionVapor,\
           T_SaturacionCondensadorFinal,U1,U2, Teb1,Teb2
    
    

def Regnault(T):
    value=606.5-0.695*T #T en ºC y value en kcal/kg
    return value


def Balances(incognitas, A, Wa_s ,Wl2_s,Ta,T1,T5,Cp,U1,U2,teb1,teb2):
    L1, L2, V1, V2, Wl1_s, W, T2, T3, T4,  Area =incognitas
    CalorT1=Regnault(T1)
    CalorT3=Regnault(T3)
    CalorT5=Regnault(T5)

    # Tutil=T1-T5-(teb1+teb2)
    #Total
    values=[A*Wa_s-L2*Wl2_s] #Ec. 1
    values.append(A-(L2+V1+V2))  #Ec. 2

    # Evaporador 1
    values.append(A-(L1+V1))  #Ec. 3
    values.append(A*Wa_s-(L1*Wl1_s))  #Ec. 4

    # Entalpico Evaporador 1
    values.append(W*CalorT1+A*Cp*(Ta-T2)-CalorT3*V1)  #Ec. 5
    # Entalpico Evaporador 2
    values.append(V1*CalorT3+L1*Cp*(T3-T4)-CalorT5*V2)  #Ec. 6


    values.append(W*CalorT1 - U1 *Area*(T1-T2))  #Ec. 7
    values.append(V1*CalorT3 - U2 * Area * (T3 - T4))  #Ec. 8
    values.append(T3-(T2-teb1))  #Ec. 9
    values.append(T5-(T4-teb2) ) #Ec. 10


    return values


display(HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Mostrar código')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Esconder código')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Mostrar código"></form>'''))

display(HTML('<h1 style="color:#000000"><strong>Avanzar para generar datos</strong></h1>'))

In [2]:
display(HTML('<h1 style="color:#000000"><strong>Los datos del sistema son:</strong></h1>'))

CaudalAlimento,W_SolutoAlimento,W_SolutoProducto,Temperatura_Alimento,T_SaturacionVapor,\
           T_SaturacionCondensadorFinal,U1,U2, Teb1,Teb2=generador_parametros()

    
valores=[np.round(CaudalAlimento),np.round((W_SolutoAlimento*100),1),np.round(Temperatura_Alimento,1),
         np.round((W_SolutoProducto*100),1),np.round(T_SaturacionVapor,1),np.round(T_SaturacionCondensadorFinal,1),
          np.round(Teb1,1),np.round(Teb2,1),np.round(U1),np.round(U2) ]


etiquetas=['A(kg/h)','Soluto_A(%)', 'Temp. A(ºC)','Soluto_L2(%)','T1(ºC)','T5(ºC)','Teb1(ºC)','Teb2(ºC)',
          'U1(kcal/h·m2·ºC)','U2(kcal/h·m2·ºC)']

data = dict(zip(etiquetas, valores))

values = pd.DataFrame(data,index=['Valores'], columns=etiquetas)


display(values)


,A(kg/h),Soluto_A(%),Temp. A(ºC),Soluto_L2(%),T1(ºC),T5(ºC),Teb1(ºC),Teb2(ºC),U1(kcal/h·m2·ºC),U2(kcal/h·m2·ºC)
Valores,22405,1.6,86.9,53.2,124.7,61.4,3.6,14.7,2784,2013


In [3]:
display(HTML('<h1 style="color:#000000"><strong>Ecuaciones balance de materia</strong></h1>' ))
display(HTML('<h2 style="color:#000000"><u>Balance Total:</u></h2>' ))
display(Math(r'A=L_2+V_1+V_2'))
display(Math(r'A\cdot X_A^s=L_2\cdot X_{L_2}^s'))

display(HTML('<h2 style="color:#000000"><u>Balance Evaporador 1:</u></h2>' ))
display(Math(r'A=L_1+V_1'))
display(Math(r'A\cdot X_A^s=L_1\cdot X_{L_1}^s'))


display(HTML('<h1 style="color:#000000"><strong>Ecuaciones balance de energía</strong></h1>' ))
display(HTML('<h2 style="color:#000000"><u>Balance Evaporador 1 ($T_{ref}=T_2$):</u></h2>' ))
display(Math(r'W\cdot \lambda_{\left ( T_1 \right )}+A\cdot C_p^{soluto}\cdot \left ( T_A-T_2 \right )\
=V_1\cdot \lambda_{\left ( T_2 \right )}'))

display(HTML('<h2 style="color:#000000"><u>Balance Evaporador 2 ($T_{ref}=T_4$):</u></h2>' ))
display(Math(r'V_1\cdot \lambda_{\left ( T_2 \right )}+L_1\cdot C_p^{soluto}\cdot \left ( T_3-T_4 \right )\
=V_2\cdot \lambda_{\left ( T_4 \right )}'))


display(HTML('<h1 style="color:#000000"><strong>Caudales de calor intercambiados</strong></h1>' ))
display(HTML('<h2 style="color:#000000"><u>Caudal de calor Evaporador 1:</u></h2>' ))
display(Math(r'W\cdot \lambda_{\left ( T_1 \right )}=U_1\cdot Area\cdot \left ( T_1-T_2 \right )'))

display(HTML('<h2 style="color:#000000"><u>Caudal de calor Evaporador 2:</u></h2>' ))
display(Math(r'V_1\cdot \lambda_{\left ( T_2 \right )}=U_2\cdot Area\cdot \left ( T_3-T_4 \right )'))


display(HTML('<h1 style="color:#000000"><strong>Temperaturas de sobrecalentamiento</strong></h1>' ))
display(HTML('<h2 style="color:#000000"><u>Vapor Evaporador 1:</u></h2>' ))
display(Math(r'T_3=T_2-T_{teb_1}'))
display(HTML('<h2 style="color:#000000"><u>Vapor Evaporador 2:</u></h2>' ))
display(Math(r'T_5=T_4-T_{teb_2}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
display(HTML('<h1 style="color:#000000"><strong>El resultado de las corrientes del sistema y del área \
de cada efecto es:</strong></h1>' ))

Resultado = optimize.fsolve(Balances, [1000, 1000, 1000, 1000, 0.9, 1000, 50, 50, 50,  50],\
                                args=(CaudalAlimento, W_SolutoAlimento ,W_SolutoProducto,Temperatura_Alimento, \
                                T_SaturacionVapor,T_SaturacionCondensadorFinal,Cp_disolucion,U1,U2, \
                                      Teb1,Teb2), xtol=1e-06, maxfev=500)


L1, L2, V1, V2, Wl1_s, W, T2, T3, T4,  Area=Resultado




    
valores_resultado=[int(np.round(L1)),int(np.round(L2)),int(np.round(V1)),int(np.round(V2)),
                   float((np.round(Wl1_s,3)*100)),int(np.round(W)),float(np.round(T2,1)),
                    float(np.round(T3,1)),float(np.round(T4,1)),int(np.round(Area))]

#print (valores_resultado)

etiquetas_resultado=['L1(kg/h)','L2(kg/h)','V1(kg/h)','V2(kg/h)','Soluto_L1(%)','W(kg/h)','T2(ºC)',
                    'T3(ºC)','T4(ºC)','Area(m2)']

data_resultado = dict(zip(etiquetas_resultado,valores_resultado))


values_resultado = pd.DataFrame(data_resultado ,index=['Valores'], columns=etiquetas_resultado)


display(values_resultado)


'''


print ("\n L1(kg/h):", int(round(L1)), " L2(kg/h):",int(round(L2))," V1(kg/h):", int(round(V1)),\
       " V2(kg/h):", int(round(V2))," Soluto_L1(%):", (round(Wl1_s,3)*100), "\n W(kg/h):", int(round(W)),\
       " T2(ºC):", round(T2,1), " T3(ºC):", round(T3,1), " T4(ºC):", round(T4,1)," Area(m2):",round(Area))''';

,L1(kg/h),L2(kg/h),V1(kg/h),V2(kg/h),Soluto_L1(%),W(kg/h),T2(ºC),T3(ºC),T4(ºC),Area(m2)
Valores,11530,674,10875,10856,3.1,11996,105.1,101.5,76.1,114


In [5]:
display(HTML('''

<footer id="attribution" style="float:right; color:#999; background:#fff;">
Programado con Jupyter Notebook en Python 3.6. </footer>'''))